In [1]:
import json
import glob
import os

create_data = False

def extract_data(dir_path):
    data = []
    for _, dirs, _ in os.walk(dir_path):
        for dir in dirs:
            files = glob.glob(os.path.join(dir_path, dir, '*.json'))
            for file in files:
                with open(file, 'r') as f:
                    content = json.load(f)
                    reliable = content['labeledDataInfo']['clickbaitClass']
                    title = content['sourceDataInfo']['newsTitle'] if reliable else content['labeledDataInfo']['newTitle']
                    content = content['sourceDataInfo']['newsContent']
                    content = data['content']
                    content = content.replace("\n", " ")
                    content = content.replace("\\\'", "\'")
                    content = content.replace('\\\"', '\"')
                    content = content.replace("\\", "")
                    data.append({
                        'title': title,
                        'content': content,
                        'reliable': reliable
                    })
    return data

if create_data:
    train_data = extract_data('../data/cls/training_set_mini') 
    test_data = extract_data('../data/cls/validation_set_mini')

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

batch_size = 8

class CBDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data = self.data[idx]
        return data['title'], data['content'], data['reliable']

if create_data:
    train_dataloader = DataLoader(CBDataset(train_data), batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(CBDataset(test_data), batch_size=batch_size)
    torch.save({'train': train_dataloader, 'test': test_dataloader},'../data/cls_data_loader.pt')
    
else:
    loaded = torch.load('../data/cls_data_loader.pt')
    train_dataloader = loaded['train']
    test_dataloader = loaded['test']

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2).cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=1e-5)

itr = 1
p_itr = 1000
epochs = 1
total_loss = 0
total_len = 0

model.train()
for epoch in range(epochs):

    for step, data in enumerate(tqdm(train_dataloader)):
        title, content, label = data
        inputs = tokenizer(title, content, padding=True, truncation=True, max_length=512, return_tensors="pt")
        inputs, label = inputs.to('cuda'), label.to('cuda')
        outputs = model(**inputs, labels=label)

        loss = outputs.loss
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % 1000 == 0:
            print('[epoch {}/{}] step {} -> loss: {:.4f}'.format(epoch+1, epochs, step+1, total_loss/(step+1)))
            torch.save(model.state_dict(), 'cls.pt')
            total_loss = 0
            total_len = 0
            
torch.save(model.state_dict(), 'cls.pt')
# Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.

  1%|          | 62/11869 [00:13<37:48,  5.20it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|          | 112/11869 [00:24<42:49,  4.57it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been remo

[epoch 1/1] step 1000 -> loss: 0.4742


  9%|▊         | 1035/11869 [03:33<39:48,  4.54it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 11%|█         | 1251/11869 [04:18<37:13,  4.75it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been r

[epoch 1/1] step 2000 -> loss: 0.1750


 18%|█▊        | 2087/11869 [07:12<32:58,  4.94it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 21%|██        | 2459/11869 [08:28<31:48,  4.93it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been r

...

 80%|███████▉  | 9448/11869 [32:44<08:40,  4.65it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 80%|███████▉  | 9457/11869 [32:46<08:35,  4.68it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been r

[epoch 1/1] step 10000 -> loss: 0.0167


 87%|████████▋ | 10327/11869 [35:48<05:22,  4.79it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 87%|████████▋ | 10362/11869 [35:56<05:39,  4.44it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been

[epoch 1/1] step 11000 -> loss: 0.0134


 93%|█████████▎| 11020/11869 [38:12<03:15,  4.33it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 93%|█████████▎| 11086/11869 [38:27<02:49,  4.63it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been

In [5]:
model.eval()
total_correct = 0

for step, data in enumerate(tqdm(test_dataloader)):
    title, content, label = data
    inputs = tokenizer(title, content, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs, label = inputs.to('cuda'), label.to('cuda')
    outputs = model(**inputs)

    logits = outputs.logits
    pred = torch.argmax(logits, dim=-1)
    correct = pred.eq(label)
    total_correct += correct.sum().item()
    total_len += len(label)

print('Test accuracy: ', total_correct / total_len)


 22%|██▏       | 99/450 [00:06<00:29, 11.81it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
 23%|██▎       | 105/450 [00:07<00:32, 10.75it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.

Test accuracy:  0.7337409672040022
